In [1]:
import os

In [2]:
pwd

'f:\\projects\\project-4 chicken disease predictor\\cd predict\\chicken-project\\reasearch'

In [3]:
os.chdir("../")

In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class TrainingConfig:
    root_dir:Path
    trained_model_path:Path
    updated_base_model_path:Path
    training_data: Path
    params_epochs:int
    params_learning_rate:float
    params_batch_size:int
    params_is_augmentation:bool
    params_image_size: list
    
@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_rot_log_dir:Path
    checkpoint_model_filepath:Path

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [24]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
            config = self.config.prepare_callbacks
            model_chkt_dir = os.path.dirname(config.checkpoint_model_filepath)
            create_directories([
                Path(model_chkt_dir),
                Path(config.tensorboard_root_log_dir)
            ])
            prepare_callback_config = PrepareCallbacksConfig(
                root_dir = Path(config.root_dir),
                tensorboard_rot_log_dir= Path(config.tensorboard_root_log_dir),
                checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
            )
            
            return prepare_callback_config
        
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"data_ingestion")
        create_directories([
            Path(training.root_dir)
        ])
        
        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_learning_rate = params.LEARNING_RATE,
            params_epochs= params.EPOCHS,
            params_batch_size= params.BATCH_SIZE,
            params_is_augmentation= params.AUGEMENTATION,
            params_image_size=params.IMAGE_SIZE
            
        )
        return training_config

In [16]:
import time
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
tf.config.run_functions_eagerly(True)  # Optional: helps debug data issues

In [8]:
class PrepareCallback:
    def __init__(self,config: PrepareCallbacksConfig):
        self.config = config
        
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y.%m.%d.%H.%M.%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_rot_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)
    
    
    @property
    def _create_cktp_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )
        
    def get_tb_ckpt_callback(self):
        return[
            self._create_tb_callbacks,
            self._create_cktp_callback
        ]

In [25]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
        self.model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=["accuracy"]
    )
    def train_valid_generator(self):
        datagen_kwargs = dict(rescale=1./255, validation_split=0.20)
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            class_mode = "categorical",
        )
        valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
        self.valid_generator = valid_datagen.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = True,
            **dataflow_kwargs
        )
        if self.config.params_is_augmentation:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=20,
                horizontal_flip=True,
                shear_range=0.2,
                zoom_range=0.2,
                **datagen_kwargs
            )
        else:
            train_datagen = valid_datagen
            
        self.train_generator = train_datagen.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )
        
    @staticmethod
    def save_model(model:tf.keras.Model, path:Path)->None:
        model.save(path)
    
    def train(self, callback_list:list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
            
        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            validation_data = self.valid_generator,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            callbacks = callback_list
        )
            
        self.save_model(self.model, self.config.trained_model_path)

In [26]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callback()
    
    training_config = config.get_training_config()
    training = Training(config = training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list=callback_list)
except Exception as e:
    raise e

[2025-12-02 17:09:08,403: INFO: common: yaml file: config\config.yaml loaded successfully]


[2025-12-02 17:09:08,448: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-02 17:09:08,468: INFO: common: created directory at : artifacts]
[2025-12-02 17:09:08,481: INFO: common: created directory at : artifacts\prepare_callbacks\checkpoint_dir]
[2025-12-02 17:09:08,488: INFO: common: created directory at : artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-12-02 17:09:08,521: INFO: common: created directory at : artifacts\training]
[2025-12-02 17:09:11,356: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 10 images belonging to 2 classes.
Found 42 images belonging to 2 classes.


c:\Users\MY LENOVO\anaconda3\envs\chicken\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\MY LENOVO\anaconda3\envs\chicken\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36s/step - accuracy: 0.4183 - loss: 5.6588  [2025-12-02 17:11:25,417: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
2/2 ━━━━━━━━━━━━━━━━━━━━ 127s 58s/step - accuracy: 0.4615 - loss: 10.4113 - val_accuracy: 0.6000 - val_loss: 18.7802
Epoch 2/5
1/2 ━━━━━━━━━━━━━━━━━━━━ 1:29 90s/step - accuracy: 0.5000 - loss: 27.0600

c:\Users\MY LENOVO\anaconda3\envs\chicken\lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


ResourceExhaustedError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} MemoryError: 
Traceback (most recent call last):

  File "c:\Users\MY LENOVO\anaconda3\envs\chicken\lib\site-packages\tensorflow\python\ops\script_ops.py", line 269, in __call__
    ret = func(*args)

  File "c:\Users\MY LENOVO\anaconda3\envs\chicken\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "c:\Users\MY LENOVO\anaconda3\envs\chicken\lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "c:\Users\MY LENOVO\anaconda3\envs\chicken\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py", line 264, in _finite_generator
    yield self._standardize_batch(self.py_dataset[i])

  File "c:\Users\MY LENOVO\anaconda3\envs\chicken\lib\site-packages\keras\sr